In [1]:
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

In [2]:
data = pd.read_csv('data/btc_data.csv', index_col=0)

In [3]:
data_crop = data.dropna()

In [4]:
# Sélectionner les caractéristiques et exclure la dernière ligne
features = data_crop.drop(columns=['progression tomorrow', 'target', 'close', 'high', 'low', 'volumefrom']).iloc[:-1, :]
target = data['target'].iloc[:-1]

In [5]:
len(data_crop)

3825

In [7]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import sys
import os

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [8]:
"""# 1. Préparation des données

X = features.values
y = target.values

window_size = 1000 # Vous pouvez ajuster cette valeur selon vos besoins

# 2. Construction du modèle

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Entraînement du modèle avec validation walk-forward

for i in range(window_size, len(X) - 1):
    # Séparation des données en ensembles d'entraînement et de test
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+1]
    y_test = y[i:i+1]

    # Normalisation des caractéristiques
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entraînement du modèle
    model.fit(X_train, y_train, epochs=10, verbose=0)  # Vous pouvez ajuster le nombre d'epochs selon vos besoins

    # Évaluation du modèle (optionnel)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Step {i}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
"""

Step 1000, Loss: 0.6367, Accuracy: 1.0000
Step 1001, Loss: 0.6922, Accuracy: 1.0000
Step 1002, Loss: 0.6472, Accuracy: 1.0000
Step 1003, Loss: 0.8133, Accuracy: 0.0000
Step 1004, Loss: 0.9729, Accuracy: 0.0000
Step 1005, Loss: 0.7805, Accuracy: 0.0000
Step 1006, Loss: 0.4903, Accuracy: 1.0000
Step 1007, Loss: 0.7328, Accuracy: 0.0000
Step 1008, Loss: 0.6660, Accuracy: 1.0000
Step 1009, Loss: 0.3724, Accuracy: 1.0000
Step 1010, Loss: 0.7476, Accuracy: 0.0000
Step 1011, Loss: 0.4768, Accuracy: 1.0000
Step 1012, Loss: 0.0627, Accuracy: 1.0000
Step 1013, Loss: 0.5891, Accuracy: 1.0000
Step 1014, Loss: 0.7323, Accuracy: 0.0000
Step 1015, Loss: 0.8181, Accuracy: 0.0000
Step 1016, Loss: 0.5359, Accuracy: 1.0000
Step 1017, Loss: 0.8902, Accuracy: 0.0000
Step 1018, Loss: 0.1858, Accuracy: 1.0000
Step 1019, Loss: 0.9002, Accuracy: 0.0000
Step 1020, Loss: 0.8163, Accuracy: 0.0000


KeyboardInterrupt: 

In [9]:
# 1. Définir la métrique personnalisée

def earn_metric(predicted_probs, progressions, n_days):
    base = c = 1
    for j in range(n_days):
        index = len(predicted_probs) - n_days + j
        c *= predicted_probs[index] * progressions[index] + (1 - predicted_probs[index])
        base *= progressions[index]
    return c / base

In [ ]:
"""
# Préparation des données

X = features.values
y = target.values

window_size = 2000  
n_days = 30 

# Construction du modèle

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Entraînement du modèle avec validation walk-forward

predicted_probs = []
progressions = []
metrics = []

for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    with HiddenPrints():
        # Entraînement
        model.fit(X_train, y_train, epochs=20, verbose=0)

        # Prédiction
        predicted_prob = model.predict(X_test).flatten()
        predicted_probs.extend(predicted_prob)

    # Récupérer la progression réelle
    progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
    progressions.extend(progression)

    # Calcul de la métrique personnalisée
    if len(predicted_probs) >= n_days:
        metric_value = earn_metric(predicted_probs, progressions, n_days)
        metrics.append(metric_value)
        #print(f"Step {i}, Earn Metric: {metric_value:.4f}")

print(f"Average Earn Metric: {np.mean(metrics):.4f}")"""

Average Earn Metric: 0.9799


In [ ]:
#Average Earn Metric: 0.9799

In [11]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

# Préparation des données

X = features.values
y = target.values

window_size = 3000  
n_days = 30 

# Construction du modèle LSTM

model = keras.Sequential([
    layers.InputLayer(input_shape=(X.shape[1], 1)),  # Les LSTM attendent une entrée tridimensionnelle (batch_size, timesteps, features)
    layers.LSTM(64, return_sequences=True),  # Retourne une séquence à la couche suivante
    layers.LSTM(32),  # Retourne la dernière sortie de la séquence
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Entraînement du modèle avec validation walk-forward

predicted_probs = []
progressions = []
metrics = []

for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Les LSTM attendent une entrée tridimensionnelle, donc nous devons remodeler les données
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    with HiddenPrints():
        # Entraînement
        model.fit(X_train, y_train, epochs=10, verbose=0)

        # Prédiction
        predicted_prob = model.predict(X_test).flatten()
        predicted_probs.extend(predicted_prob)

    # Récupérer la progression réelle
    progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
    progressions.extend(progression)

    # Calcul de la métrique personnalisée
    if len(predicted_probs) >= n_days:
        metric_value = earn_metric(predicted_probs, progressions, n_days)
        metrics.append(metric_value)

print(f"Average Earn Metric: {np.mean(metrics):.4f}")

In [ ]:
import autokeras as ak
from sklearn.preprocessing import StandardScaler
import sys
import os

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Préparation des données
X = features.values
y = target.values

window_size = 3000  
n_days = 30 

# Création du modèle AutoKeras
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=10,  # Nombre d'architectures différentes à essayer
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Entraînement du modèle avec validation walk-forward
predicted_probs = []
progressions = []
metrics = []

for i in range(window_size, len(X) - 1 - n_days):
    # Séparation des données
    X_train = X[i-window_size:i]
    y_train = y[i-window_size:i]
    X_test = X[i:i+n_days]
    y_test = y[i:i+n_days]

    # Normalisation
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    with HiddenPrints():
        # Entraînement
        clf.fit(X_train, y_train, epochs=10)

        # Prédiction
        predicted_prob = clf.predict(X_test).flatten()
        predicted_probs.extend(predicted_prob)

    # Récupérer la progression réelle
    progression = data_crop.iloc[i:i+n_days]['progression tomorrow'].values + 1
    progressions.extend(progression)

    # Calcul de la métrique personnalisée
    if len(predicted_probs) >= n_days:
        metric_value = earn_metric(predicted_probs, progressions, n_days)
        metrics.append(metric_value)

print(f"Average Earn Metric: {np.mean(metrics):.4f}")